# WebsocketServerWorker Tutorial

This tutorial is a 2 notebook tutorial. The partner notebook is the notebook entitled SocketWorker Client.ipynb and is in the same folder as this notebook. You should execute this notebook BEFORE the other.

In this tutorial, we'll demonstrate how to launch a WebsocketWorker server which will listen for PyTorch commands over a socket connection. In this tutorial, the two workers are connected via a socket connection on the localhost network.

If you'd prefer to download this notebook and run it locally on your machine, you can do so via the `Download .ipynb` in the `File` dropdown field in this Google colab environment. 


# Step -1: Copy This Notebook

Go up to File -> Save A Copy in Drive

This will let you execute the notebook (it won't let you execute this one by default)

# Step 0: Install Dependencies

In [4]:
! git clone https://github.com/OpenMined/PySyft.git
! git checkout dev
! git pull origin dev
# http://pytorch.org/
from os.path import exists
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'

#!pip3 install https://download.pytorch.org/whl/cu100/torch-1.1.0-cp36-cp36m-linux_x86_64.whl
#!pip3 install https://download.pytorch.org/whl/cu100/torchvision-0.3.0-cp36-cp36m-linux_x86_64.whl

import torch

!cd PySyft; pip install -r requirements.txt; python setup.py install

import os
import sys
print(sys.path)

fatal: destination path 'PySyft' already exists and is not an empty directory.
Already on 'dev'
Your branch is behind 'origin/dev' by 5 commits, and can be fast-forwarded.
  (use "git pull" to update your local branch)
From https://github.com/OpenMined/PySyft
 * branch              dev        -> FETCH_HEAD
Updating 9965dad8..2d97e7b6
Fast-forward
 syft/__init__.py                           | 6 +++++-
 syft/frameworks/torch/federated/dataset.py | 6 +++---
 2 files changed, 8 insertions(+), 4 deletions(-)
running install
running bdist_egg
running egg_info
writing syft.egg-info/PKG-INFO
writing dependency_links to syft.egg-info/dependency_links.txt
writing requirements to syft.egg-info/requires.txt
writing top-level names to syft.egg-info/top_level.txt
reading manifest file 'syft.egg-info/SOURCES.txt'
writing manifest file 'syft.egg-info/SOURCES.txt'
installing library code to build/bdist.macosx-10.9-x86_64/egg
running install_lib
running build_py
creating build/bdist.macosx-10.9-x86_64/e

In [5]:
import sys
print(sys.path)

['/Users/kastern/PySyft/examples/tutorials', '/anaconda3/envs/pysyft/lib/python37.zip', '/anaconda3/envs/pysyft/lib/python3.7', '/anaconda3/envs/pysyft/lib/python3.7/lib-dynload', '', '/anaconda3/envs/pysyft/lib/python3.7/site-packages', '/anaconda3/envs/pysyft/lib/python3.7/site-packages/IPython/extensions', '/Users/kastern/.ipython']


In [12]:
#sys.path.append('./PySyft')
print(sys.path)

['/Users/kastern/PySyft/examples/tutorials', '/anaconda3/envs/pysyft/lib/python37.zip', '/anaconda3/envs/pysyft/lib/python3.7', '/anaconda3/envs/pysyft/lib/python3.7/lib-dynload', '', '/anaconda3/envs/pysyft/lib/python3.7/site-packages', '/anaconda3/envs/pysyft/lib/python3.7/site-packages/IPython/extensions', '/Users/kastern/.ipython', './PySyft']


# Step 1: Hook PyTorch

Just like previous tutorials, the first step is to override PyTorch commands using the TorchHook object.

In [13]:
import syft as sy

hook = sy.TorchHook(torch)

W0724 02:07:51.394925 4448884160 hook.py:98] Torch was already hooked... skipping hooking process


# Step 2: Launch Server

The next step is to launch the server. We set the `host` to "localhost", `hook` to the "hook" set above, `id` to the "id" of the worker, and `port` to the worker's "port" (in this case the port is at `8182`). 

In [14]:
from syft.workers.websocket_server import WebsocketServerWorker

local_worker = WebsocketServerWorker(
                            host="localhost",
                            hook=hook,
                            id=0,
                            port=8182)

local_worker.start()  # Might need to interrupt with `control-C` or some other means

RuntimeError: This event loop is already running

In [15]:
local_worker.list_objects()

'{}'

In [16]:
local_worker.objects_count()

0

In [17]:
local_worker.port

8182

W0724 02:20:39.370642 4448884160 base.py:577] Worker me couldn't recognize worker 2
W0724 02:20:39.376957 4448884160 base.py:577] Worker me couldn't recognize worker 2
E0724 02:20:39.380067 4448884160 base_events.py:1608] Task exception was never retrieved
future: <Task finished coro=<WebsocketServerWorker._producer_handler() done, defined at ./PySyft/syft/workers/websocket_server.py:89> exception=AttributeError("'int' object has no attribute 'id'")>
Traceback (most recent call last):
  File "./PySyft/syft/workers/websocket_server.py", line 107, in _producer_handler
    response = self._recv_msg(message)
  File "./PySyft/syft/workers/websocket_server.py", line 118, in _recv_msg
    return self.recv_msg(message)
  File "./PySyft/syft/workers/base.py", line 254, in recv_msg
    response = self._message_router[msg_type](contents)
  File "./PySyft/syft/workers/base.py", line 365, in execute_command
    response = getattr(_self, command_name)(*args, **kwargs)
  File "./PySyft/syft/framework